<center>

**Vježba 6, korak 1 - Vremenski skup podataka i PyTorch LSTM arhitekture**

</center>

U ovoj vježbi želimo vidjeti kako modelirati vremenske slijedove podataka. U tu svrhu koristiti ćemo rekurzivnu LSTM neuronsku mrežu iz modula PyTorch.

Sljedeći skup podataka možete skinuti ako CSV datoteka ne postoji u radnom direktoriju. Skup podataka predstavlja vremenski slijed putnika određene zrakoplovne kompanije.

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv

Učitavamo nužne python module, funkcije i klase.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

Učitavamo vremensku seriju, koja je prilično jednostavna i dvodimenzionalna. Imamo vremensku dimenziju i broj putnik koji predstavlja drugu dimenziju. U dijagram koji dobijemo vizualizira se ovaj vremenski slijed podataka.

Primjećujemo jaku pravilnost (skoro da se može nazvati sezonalnošću) broja putnika.

In [ ]:
train_set1 = pd.read_csv('airline-passengers.csv')
train_set2 = train_set1.iloc[:,1:2].values

plt.plot(train_set2, label = 'Airline Passengers Number')
plt.show()

Definiramo funkciju za kližući prozor. Na taj se način definira prozor koji kliže kroz vrijeme, ograničujući tako vremensku seriju na taj prozor. Treba primijetiti da je ciljna varijabla podešena tako da spremi broj putnika prvu seriju nakon vremenskog prozora. Na taj se način LSTM uči predviđanju (*forecasting*).

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

Prvo skaliramo podatke iz ulazne vremenske serije. U tu svrhu koristimo ```MinMaxScaler``` koji sve vrijednosti smješta između minimalne (0) i maksimalne (1).  
[MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

Zatim razdjeljujemo vremensku seriju na podseriju za učenje i poseriju za testiranje. Ovo je nezgodno jer se ne smije desiti da izmiješamo vremensku seriju, ona u vremenskoj dimenziji mora ostati netaknuta. Jedini način je da se da iz vremenske serije izdvoje sve instance kližućeg prozora, pa se dio tih instance izdvoji za testiranje (u našem slučaju imamo 67% za učenje).

In [ ]:

sc = MinMaxScaler()
training_data = sc.fit_transform(train_set2)

seq_length = 4
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

Slijedi definiranje arhitekture neuronske mreže za Pytorch.

Naša arhitektura ima:
1) LSTM dio koji prima jednu značajku po neuronu. Broj neurona ovisi o veličini kližućeg prozora.
2) Skriveni dio LSTM-a spaja se na FC (*fully connected*) izlaz. Skriveni dio ima 2 značajke. Izlaz ima jednu "klasu". S obzirom da ne koristimo aktivacijsku funkciju, izlaz će biti kontinuirane prirode.

[Fully connected layer](https://docs.nvidia.com/deeplearning/performance/dl-performance-fully-connected/index.html)

In [ ]:
class AirlineLSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(AirlineLSTM, self).__init__()        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))        
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        _, (h_out, _) = self.lstm(x, (h0, c0))        
        h_out = h_out.view(-1, self.hidden_size)        
        out = self.fc(h_out)        
        return out

Definiramo maksimalni broj epoha na 2000.

Ulaz je jedna značajka, broj skrivenih značajki u LSTM sloju je 2, broj LSTM slojeva je 1 i broj izlaza iz FC je 1.

Stvorimo prethodno opisanu neuronsku mrežu.

Kriterij za izračun gubitka je *Mean Square Error* koji je uobičajen za kontinuirane izlaze.
Optimizator za gradijente je *Adam*.

U petlji prolazimo podskup instanci prozora za učenje, računamo gubitak i radimo optimizaciju.
Nakon petlje smatramo da smo naučili vremensku seriju.

In [ ]:
num_epochs = 4000
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1
num_classes = 1

lstm = AirlineLSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    loss = criterion(outputs, trainY)    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

Testiramo našu arhitekturu neuronske mreže testnim podskupom instanci prozora.
Potrebna je transformacija na tenzore koji se koriste u PyTorch modulu.

Iscrtavamo testne vrijednosti određene od strane neuronske mreže (*predict*) i prave testne vrijednosti (*actual*).

In [ ]:
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.title('Prediction')
plt.show()

Hiperparametre PyTorch modela spremimo u datoteku. Primijetimo da su ovo samo hiperparametri i da datoteka neće sadržavati arhitekturu neuronske mreže.

In [ ]:
torch.save(lstm.state_dict(), 'airline.pth')

print(lstm.state_dict())